In [3]:
import poke
import sys
import numpy as np
# Code V Imports for com interface
from pythoncom import (CoInitializeEx, CoUninitialize, COINIT_MULTITHREADED, com_error )
from pythoncom import VT_VARIANT, VT_BYREF, VT_ARRAY, VT_R8
from win32com.client import DispatchWithEvents, Dispatch, gencache, VARIANT
from win32com.client import constants as c  # To use enumerated constants from the COM object typelib
from win32api import FormatMessage

sys.coinit_flags = COINIT_MULTITHREADED
dir = "c:\cvuser"

In [5]:
class ICVApplicationEvents:
    def OnLicenseError(self, error):
        # This event handler is called when a licensing error is 
        # detected in the CODE V application.
        print ("License error: %s " % error)

    def OnCodeVError(self, error):
        # This event handler is called when a CODE V error message is issued
        print ("CODE V error: %s " % error)

    def OnCodeVWarning(self, warning):
        # This event handler is called when a CODE V warning message is issued
        print ("CODE V warning: %s " % warning)

    def OnPlotReady(self, filename, plotwindow):
        # This event handler is called when a plot file, refered to by filename,
        # is ready to be displayed.
        # The event handler is responsible for saving/copying the
        # plot data out of the file specified by filename
        print ("CODE V Plot: %s in plot window %d" % (filename ,plotwindow) )

def Command(s):
    
    cv.Command(s)
    
def boilerplate_cvapi():
    
    cv = DispatchWithEvents("CodeV.Application", ICVApplicationEvents)
    
    directory="c:\\cvuser"
    cv.StartingDirectory = directory
    cv.StartCodeV()
    
    return cv
    
    
    

def raytrace_cv():
    
    """
    x,y are ray coordinates at s0. Want them returned at ssurf
    """
    
    cv = DispatchWithEvents("CodeV.Application", ICVApplicationEvents)
    
    directory="c:\\cvuser"
    cv.StartingDirectory = directory
    cv.StartCodeV()

    # Generate some warning events
    result = cv.Command("res cv_lens:ag_dblgauss")
    print(result)
    
    def Command(s):
    
        cv.Command(s)
    
    nZoomPos       = 1
    nWavelengthNum = 1
    nFieldNum      = 2
    nRefSurf       = 1
    saRAYRSI = np.zeros(4)

    saRAYRSI[0] = 0.0
    saRAYRSI[1] = 1.0
    saRAYRSI[2] = 0.0
    saRAYRSI[3] = 1.0
    # Convert the generated numpy array into a form that can be used by COM
    listRAYRSI = np.ascontiguousarray(saRAYRSI).tolist()

#         print("(AOI Si) ="+cv.EvaluateExpression("(AOI SI)"))
    dResult = cv.RAYRSI(nZoomPos, nWavelengthNum, nFieldNum, nRefSurf, listRAYRSI)
#         print("RAYRSI> " + str(dResult))
#         print("(AOI Si) ="+ cv.EvaluateExpression("(AOI SI)"))

# Using Polgrid

Polgrid looks like the most efficient method to do the polarization ray tracing. It forgoes Poke entirely, but we can still use it to parse the data. We could even check poke against Polgrid.

The polgrid inputs are
`POLGRID(zoom_pos, wave_num, field_pos, aper_check, nrays, ^input_array, ^output_array)`
- `zoom_pos` is the zoom position
- `wave_num` is the wavelength
- `field_pos` is the FoV to trace
- `aper_check` Flag to check apertures. 0 means do not check apertures, non-zero means to check apertures. I don't know what "check apertures" means, but I assumes this looks to see if the rays are vignetted
- `nrays` is the number of rays in the grid
- `^input_array` is a 2xN (where N = nrays) element input array containing the relative pupil coordinates (x,y) for each of the rays to be traced in the grid.
- `^output_array` A two-dimensional (i,j) output array. The ith array index can accept values between 1 and 43, corresponding to the output data items in the numbered list below. The jth array index identifies the ray that corresponds to each set of output values, and can accept values between 1 and (N+1), where N is the desired number of grid rays. j=1 corresponds to the chief ray, and the remaining values (2 through (N+1)) correspond to the grid rays specified with the input array. Elements are as follows.

There are 43 outputs, of which I will not list all of them. But I will list the ones relevant to PRT calculations. They only return at image surface so unfortunately we have to rely entirely on code v.

^output_array (21) –  Real part of XX component of Jones matrix (JR1). 
^output_array (22) –  Imaginary part of XX component of Jones matrix (JI1). 
^output_array (23) –  Real part of XY component of Jones matrix (JR2). 
^output_array (24) –  Imaginary part of XY component of Jones matrix (JI2). 
^output_array (25) –  Real part of YX component of Jones matrix (JR3). 
^output_array (26) –  Imaginary part of YX component of Jones matrix (JI3). 
^output_array (27) –  Real part of YY component of Jones matrix (JR4). 
^output_array (28) –  Imaginary part of YY component of Jones matrix (JI4). 


In [11]:
def run_polgrid():
    
    cv = boilerplate_cvapi()
    
    def Command(s):
        cv.Command(s)
        
    Command('res cv_lens:ag_dblgauss')
    Command('')
    del cv

In [12]:
run_polgrid()